In [1]:
import jieba
import re
import copy
from datetime import datetime
from pymongo import MongoClient
import pprint
import dpath.util
import pandas as pd
import gensim
import pickle

Load Data and Save Data to Mongo DB

In [ ]:
def Data_insert_MongoDB(News, collection_name):
    with open(News,"rb") as fp:
    a = pickle.load(fp)
    t = a.to_dict('records')
    client = MongoClient("mongodb://192.168.87.135:27017")
    db = client["news"]
    posts = db[collection_name]
    result = posts.insert_many(t)
    result.inserted_ids


In [ ]:
Data_insert_MongoDB("news_all_with_event_with_tags_5days","news_all_epspointseven_with_event_5days")
Data_insert_MongoDB("valid_news_all_with_event_with_tags_5days","valid_news_all_epspointseven_with_event_5days")

Making Major Event and Minor Event(Outlier) 

In [ ]:
def Event_making(collection_name):
    client = MongoClient("mongodb://192.168.87.135:27017")
    db = client["news"]
    collection = db[collection_name]
    news_list_temp = collection.aggregate(    [      { "$group" : { "_id" : {"tag":"$tag", "subtag":"$subtag"}, "dmin":{"$min":"$datetime"},"dmax":{"$max":"$datetime"},"org_indexes":{ "$addToSet" : "$org_index" }}}  ,{"$project":{"diff":{"$subtract":["$dmax","$dmin"]},"org_indexes":"$org_indexes","dmin":"$dmin","dmax":"$dmax"} }] )
    return news_list_temp

In [ ]:
news_list_temp = Event_making("news_all_epspointseven_with_event_5days")
news_list_temp_v = Event_making("valid_news_all_epspointseven_with_event_5days")

In [14]:
news_list = list(news_list_temp)
print(news_list)
news_df = pd.DataFrame(news_list)

[{'_id': {'tag': 2, 'subtag': 0}, 'diff': 0, 'org_indexes': [297, 296, 295, 294, 293, 292, 291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 281, 280, 279, 278, 253, 252, 251, 250, 249, 248, 247, 246, 245, 244, 243, 277, 242, 275, 235, 261, 234, 259, 233, 257, 232, 236, 263, 237, 265, 238, 267, 239, 269, 240, 271, 241, 273, 254, 255, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276], 'dmin': datetime.datetime(2017, 12, 14, 0, 0), 'dmax': datetime.datetime(2017, 12, 14, 0, 0)}, {'_id': {'tag': 0, 'subtag': -1}, 'diff': 0, 'org_indexes': [34], 'dmin': datetime.datetime(2017, 12, 17, 0, 0), 'dmax': datetime.datetime(2017, 12, 17, 0, 0)}, {'_id': {'tag': -1, 'subtag': -1}, 'diff': 3283200000, 'org_indexes': [302, 300, 231, 230, 229, 228, 227, 226, 224, 223, 222, 221, 220, 219, 218, 216, 215, 214, 213, 212, 211, 210, 96, 95, 299, 11, 94, 92, 91, 88, 87, 10, 86, 84, 83, 80, 79, 9, 78, 76, 75, 72, 70, 64, 199, 63, 195, 62, 191, 61, 60, 183, 59, 58, 56, 55, 54, 53, 155, 52, 151, 51, 147, 

In [ ]:
def list_to_df(news_list_temp):
    news_list = list(news_list_temp)
    news_df = pd.DataFrame(news_list)
    return news_df
news_df = list_to_df(news_list_temp)
news_df_v = list_to_df(news_list_temp_v)

In [ ]:
def transfer_tag(news_df):
    tagsubtag = list(news_df["_id"])
    tagsubtag = pd.DataFrame(tagsubtag)
    # preprocess the dataframe
    news_df["diff"] = (news_df["diff"]/(60*60*24*1000))
    news_df["tag"] = tagsubtag["tag"]
    news_df["subtag"] = tagsubtag["subtag"]
    news_df = news_df[(news_df.tag != -1) & (news_df.subtag != -1)]
    news_df = news_df.sort_values(["diff"],ascending = [0])
    return news_df
news_df = transfer_tag(news_df)
news_df_v = transfer_tag(news_df_v)
    

In [ ]:
def outlier_making(collection_name):
    client = MongoClient("mongodb://192.168.87.135:27017")
    db = client["news"]
    collection = db[collection_name]
    news_list_temp = collection.find({})
    news_list = list(news_list_temp)
    news_df_outlier = pd.DataFrame(news_list)
    news_df_outlier=news_df_outlier.rename(columns = {'org_index':'org_indexes'})
    # ---------------------------------------------
    org_indexes_tmp = []
    for index, row in news_df_outlier.iterrows():
        tmp = []
        tmp.append(row.loc["org_indexes"])
        org_indexes_tmp.append(tmp)
    org_indexes_tmp
    
    news_df_outlier["org_indexes"] = pd.Series(org_indexes_tmp)
    # --------------------------------------------
    news_df_outlier = news_df_outlier[(news_df_outlier.tag == -1) | (news_df_outlier.subtag == -1)]
    news_df_outlier.drop("_id",1,inplace=True)
    news_df_outlier["dmax"] = news_df_outlier["datetime"]
    news_df_outlier["dmin"] = news_df_outlier["datetime"]
    news_df_outlier["diff"] = 0

news_df_outlier = outlier_making("news_all_epspointseven_with_event_5days")
news_df_outlier_v = outlier_making("valid_news_all_epspointseven_with_event_5days")

In [ ]:
def merge_event_outlier(news_df,news_df_outlier):
    frames = [news_df,news_df_outlier]
    result = pd.concat(frames)
    return result

result = merge_event_outlier(news_df,news_df_outlier)
result_v = merge_event_outlier(news_df_v,news_df_outlier_v)

In [6]:
with open("Events_eps0.7_5days","wb") as fp:
    pickle.dump(result,fp)

In [22]:
with open("Valid_events_eps0.7_5days","wb") as fp:
    pickle.dump(result_v,fp)

In [25]:
result = result.sort_values(["dmin"],ascending=[1])

In [ ]:
# Checking the days distribution of Events
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid') #預設作圖樣式
import numpy as np
fig = plt.figure(figsize = (400,6)) 
ax = plt.axes()
pltx = np.array(result["dmin"])
plty = np.array(result["diff"])
ax.plot(pltx, plty)